<a href="https://colab.research.google.com/github/ReidelVichot/LC_identification/blob/main/USASpendingData_4_17_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import time

# -- this line is to make pandas future-proof, Copy-on-Write will become the default in pandas 3.0.
pd.options.mode.copy_on_write = True

# -- Set the data path
dpath = "/content/drive/MyDrive/Disertation/"
fname = "USASpendingData/PrimeTransactionsAndSubawards_2024-03-28_H20M13S03380006/Assistance_PrimeTransactions_2024-03-28_H20M13S55_1.csv"


In [4]:
df = pd.read_csv(dpath + fname, low_memory = False)

In [6]:
df.head(5)

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,...,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
0,8620_NY06C70-7013_-NONE-_14.238_-NONE-,ASST_NON_NY06C70-7013_8620,NY06C70-7013,NaN,NaN,SAI NOT AVAILABLE,247896.0,247896.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-04-30
1,8620_NY01C70-3015_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-3015_8620,NY01C70-3015,NaN,FY2011-TCIR-DOC11442607537-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,313896.0,313896.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-09-16
2,8620_NY01C70-0230_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-0230_8620,NY01C70-0230,NaN,NaN,SAI NOT AVAILABLE,192276.0,192276.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-08-08
3,8620_NY01C60-2011_-NONE-_14.238_-NONE-,ASST_NON_NY01C60-2011_8620,NY01C60-2011,NaN,FY2011-TCIR-DOC11442607531-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,84960.0,84960.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2007-12-19
4,8620_NY01C40-0153_-NONE-_14.238_-NONE-,ASST_NON_NY01C40-0153_8620,NY01C40-0153,NaN,NaN,SAI NOT AVAILABLE,-47771.0,-47771.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/ASST_NON_NY0...,2017-09-17,2008-02-26


In [ ]:
# columns of interest:
# federal_action_obligation : Amount of Federal Government's obligation,
#       de-obligation, or liability in dollars for an award transaction. It
#       is what the governemnt promises to pay to non-federal recipients.
# total_obligated_amount : Sum of all the amounts entered in the Action
#       Obligation field.
# indirect_cost_federal_share_amount : Amount of any single Federal award that
#       it is allocated to indirect costs according to the approved budget.
# generated_pragmatic_obligations
# non_federal_funding_amount: Amount of the award funded by non-Federal sources
# total_non_federal_funding_amount: The amount of the total award funded by non-
#       Federal sources.
# action_date : The date the action being reported was issued / signed by the
#       Government or a binding agreement was reached.
# action_date_fiscal_year : Note that the Federal fiscal year begins on
#       October 1 and ends on September 30, thus October 1, 2018 is the
#       first day of the 2019 fiscal year.
# period_of_performance_start_date : The Period of Performance is defined in
#       the 2 CFR 200 as the total estimated time interval between the start of
#       an initial Federal award and the planned end date, which may include
#       one or more funded portions, or budget periods.
# period_of_performance_current_end_date : The contract completion date based
#       on the schedule in the contract. For an initial award, this is the
#       scheduled completion date for the base contract and for any options
#       exercised at time of award.
# assistance_transaction_unique_key : System-generated database key used to
#       uniquely identify each financial assistance transaction record and
#       facilitate record lookup, correction, and deletion. A concatenation
#       of AwardingSubTierAgencyCode, FAIN, URI, AssistanceListingNumber, and
#       AwardModificationAmendmentNumber with a single underscore ('_')
#       character inserted in between each. If a field is blank, it is
#       recorded as "".
# recipient_address_line_1
# recipient_address_line_2
# recipient_city_code
# recipient_city_name
# prime_award_transaction_recipient_county_fips_code
# recipient_county_name
# prime_award_transaction_recipient_state_fips_code
# recipient_state_code
# recipient_state_name
# recipient_zip_code


In [15]:
cols = ['federal_action_obligation', 'total_obligated_amount',
        'indirect_cost_federal_share_amount', 'non_federal_funding_amount',
        'action_date', 'action_date_fiscal_year', 'period_of_performance_start_date',
        'period_of_performance_current_end_date', 'assistance_transaction_unique_key',
        'recipient_address_line_1', 'recipient_address_line_2', 'recipient_city_code',
        'recipient_city_name', 'prime_award_transaction_recipient_county_fips_code',
        'recipient_county_name', 'prime_award_transaction_recipient_state_fips_code',
        'recipient_state_code', 'recipient_state_name', 'recipient_zip_code']
df = df[cols]

In [12]:
colsname = ["State", "State_fips", "County_fips", "County_name", "FIPS_class"]
from_census = "https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt"
county_fips = pd.read_csv(from_census, names=colsname, header=None)
county_fips = county_fips.drop(columns="FIPS_class")
county_fips["GEOID"] =  county_fips.State_fips.astype(str).str.zfill(2) + county_fips.County_fips.astype(str).str.zfill(3)

In [33]:
# Removing Foreign investment
df = df[df["recipient_city_code"] != "FORGN"]
# Removing NaN fips codes
df = df[~df.prime_award_transaction_recipient_state_fips_code.isna()]

In [39]:
df["GEOID"] = df.prime_award_transaction_recipient_state_fips_code.astype(int).astype(str).str.zfill(2) + df.prime_award_transaction_recipient_county_fips_code.astype(int).astype(str).str.zfill(3)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [42]:
df[df.prime_award_transaction_recipient_county_fips_code.isna()]

,federal_action_obligation,total_obligated_amount,indirect_cost_federal_share_amount,non_federal_funding_amount,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,assistance_transaction_unique_key,recipient_address_line_1,recipient_address_line_2,recipient_city_code,recipient_city_name,prime_award_transaction_recipient_county_fips_code,recipient_county_name,prime_award_transaction_recipient_state_fips_code,recipient_state_code,recipient_state_name,recipient_zip_code,GEOID
468,500000.0,500000.0,NaN,0.0,2008-07-29,2008,2008-07-29,2011-07-28,6959_IT085501G00000_-NONE-_20.237_0000,WISCONSIN DEPART OF TRANSPORTATION,4802 SHEBOYGAN AVENUE,NaN,NaN,NaN,NaN,55.0,WI,WISCONSIN,53707.0,55nan
2386,48085.0,48085.0,NaN,48928.0,2009-04-03,2009,2009-07-01,2010-06-30,1434_G09AC00091_DOINBC0500_15.809_-NONE-,NaN,NaN,35600,HELENA,NaN,LEWIS AND CLARK,30.0,MT,MONTANA,59620.0,30nan
3640,2225500.0,9684120.0,NaN,0.0,2009-07-10,2009,2009-07-10,2009-07-15,1450_GTA08X331_DOIBIA08052009-00684_15.042_19,NaN,NaN,41180,MARTY,NaN,CHARLES MIX,46.0,SD,SOUTH DAKOTA,57361.0,46nan
6042,396900.0,9684120.0,NaN,0.0,2009-07-07,2009,2009-07-07,2009-07-27,1450_GTA08X331_DOIBIA08052009-00686_15.046_19,NaN,NaN,41180,MARTY,NaN,CHARLES MIX,46.0,SD,SOUTH DAKOTA,57361.0,46nan
7069,-1.0,240982.0,NaN,0.0,2009-07-09,2009,2009-07-09,2009-07-20,1450_CTE02X811_DOIBIA08052009-00201_15.04_24,NaN,NaN,3550,ANIAK,NaN,BETHEL (CA),2.0,AK,ALASKA,99557.0,02nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356095,50000.0,50000.0,NaN,36410.0,2008-09-24,2008,2008-09-19,2009-12-30,1448_701818J743_-NONE-_15.63_-NONE-,NaN,NaN,3000,ANCHORAGE,NaN,ANCHORAGE,2.0,AK,ALASKA,99519.0,02nan
356096,25096.0,25096.0,NaN,0.0,2008-08-04,2008,2008-08-04,2009-09-30,1443_J1492080039_-NONE-_15.DAV_-NONE-,NaN,NaN,23620,FLAGSTAFF,NaN,COCONINO,4.0,AZ,ARIZONA,86011.0,04nan
356097,150400.0,2581675.0,NaN,0.0,2008-09-11,2008,2008-08-11,2008-10-01,1450_GTP11T124_-NONE-_15.044_39,NaN,NaN,71960,TOPPENISH,NaN,YAKIMA,53.0,WA,WASHINGTON,98948.0,53nan
361549,24782.0,670185.0,NaN,0.0,2009-07-16,2009,2009-07-16,2010-09-30,1450_CTP06T105_DOIBIA08052009-00583_15.02_21,NaN,NaN,10950,CEDARVILLE,NaN,GRAYS HARBOR,53.0,WA,WASHINGTON,98568.0,53nan


In [ ]:
df.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,modification_number,award_id_uri,sai_number,federal_action_obligation,total_obligated_amount,total_outlayed_amount_for_overall_award,indirect_cost_federal_share_amount,...,business_types_description,correction_delete_indicator_code,correction_delete_indicator_description,action_type_code,action_type_description,record_type_code,record_type_description,highly_compensated_officer_1_name,initial_report_date,last_modified_date
0,8620_NY06C70-7013_-NONE-_14.238_-NONE-,ASST_NON_NY06C70-7013_8620,NY06C70-7013,NaN,NaN,SAI NOT AVAILABLE,247896.0,247896.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-04-30
1,8620_NY01C70-3015_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-3015_8620,NY01C70-3015,NaN,FY2011-TCIR-DOC11442607537-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,313896.0,313896.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-09-16
2,8620_NY01C70-0230_-NONE-_14.238_-NONE-,ASST_NON_NY01C70-0230_8620,NY01C70-0230,NaN,NaN,SAI NOT AVAILABLE,192276.0,192276.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2008-08-08
3,8620_NY01C60-2011_-NONE-_14.238_-NONE-,ASST_NON_NY01C60-2011_8620,NY01C60-2011,NaN,FY2011-TCIR-DOC11442607531-PDATE2011-05-24-LIN...,SAI NOT AVAILABLE,84960.0,84960.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,A,NaN,2,NaN,NaN,2017-09-17,2007-12-19
4,8620_NY01C40-0153_-NONE-_14.238_-NONE-,ASST_NON_NY01C40-0153_8620,NY01C40-0153,NaN,NaN,SAI NOT AVAILABLE,-47771.0,-47771.0,NaN,NaN,...,STATE GOVERNMENT,NaN,NaN,C,NaN,2,NaN,NaN,2017-09-17,2008-02-26


In [44]:
#!/usr/bin/env python
import json
import re

statecodes = json.load(open('state_fips.json'))
zipmap = {}

for i in range(1,11):
    zfile = open('zipctys/zipcty%d' % i)
    zfile.readline() # skip first line
    for l in zfile:
        m = re.match(r"(?P<zip>.{5}).{18}(?P<state>..)(?P<fips>...)", l)
        if m:
            r = m.groupdict()
            zipmap[r['zip']] = statecodes[r['state']] + r['fips']

print(json.dumps(zipmap))

/bin/bash: line 1: cd: https://github.com/bgruber/zip2fips.git: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'state_fips.json'